When choosing a model two approaches were given to us.Either creating our own model using TF-IDF , or using an already made model like BERT, ROBERTA etc

The first model below is a model using a TF-IDF Vectorizer and Naive Bayes—a classic approach..

# STEP 4-TF IDF AND NAIVE BAYES

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Charger les données
import pandas as pd
df = pd.read_csv("train_cleaned_numeric_labels.csv")

# Transformer les textes en vecteurs TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X_tfidf = vectorizer.fit_transform(df["Text"])

# Séparer les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df["Label"], test_size=0.2, random_state=42)

# Entraîner le modèle Naive Bayes
model_nb = MultinomialNB()
model_nb.fit(X_train, y_train)

# Prédictions
y_pred = model_nb.predict(X_test)

# Évaluer l'accuracy
print(f"Accuracy du modèle Naive Bayes : {accuracy_score(y_test, y_pred):.4f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.model_selection import train_test_split

# Charger les données
df = pd.read_csv("train_cleaned_numeric_labels.csv")

# Tokenisation des textes
tokenizer = Tokenizer(num_words=50000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["Text"])
X_seq = tokenizer.texts_to_sequences(df["Text"])

# Padding des séquences
X_pad = pad_sequences(X_seq, maxlen=200, padding='post', truncating='post')

# Séparer les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X_pad, df["Label"], test_size=0.2, random_state=42)

# Création du modèle LSTM
model_lstm = Sequential([
    Embedding(input_dim=50000, output_dim=128, input_length=200),
    SpatialDropout1D(0.2),
    LSTM(128, return_sequences=False),
    Dense(128, activation="relu"),
    Dense(len(df["Label"].unique()), activation="softmax")
])

# Compilation du modèle
model_lstm.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Entraînement
model_lstm.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Évaluation
loss, acc = model_lstm.evaluate(X_test, y_test)
print(f"Accuracy du modèle LSTM : {acc:.4f}")


# STEP 5-Pre-existing Model

The code below is our first attempt at using models.

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Charger le tokenizer et le modèle BERT multilingue
model_name = 'bert-base-multilingual-cased'  # ou 'xlm-roberta-base' pour XLM-RoBERTa
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=386)  # 386 labels pour tes 386 langues

# Charger le dataset
import pandas as pd
df = pd.read_csv("train_cleaned_numeric_labels.csv")

# Tokenisation des textes
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding='max_length', truncation=True, max_length=512)

# Appliquer la tokenisation au dataset
tokenized_dataset = df[['Text', 'Label']].rename(columns={'Text': 'text', 'Label': 'label'})
tokenized_dataset = tokenized_dataset.map(tokenize_function, batched=True)

# Diviser le dataset en train/test
train_dataset, test_dataset = train_test_split(tokenized_dataset, test_size=0.2, random_state=42)

# Convertir en Dataset Hugging Face
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

# Définir les arguments pour l'entraînement
training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy="epoch",     
    learning_rate=2e-5,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    num_train_epochs=3,             
    weight_decay=0.01,              
    logging_dir='./logs',            
    logging_steps=200,
)

# Fonction d'évaluation pour calculer l'accuracy
def compute_metrics(p):
    predictions, labels = p
    preds = torch.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(labels, preds)}

# Créer le Trainer Hugging Face
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,            
    compute_metrics=compute_metrics       
)

# Entraîner le modèle
trainer.train()

# Évaluer le modèle
results = trainer.evaluate()
print(f"Accuracy sur le dataset de test : {results['eval_accuracy']:.4f}")



# THE CODE WITH AN ACCURACY OF 0.73

The result of our code is in the .ipynb named : Models.ipynb (we ran it on another file so in order to see the results we needed to split it ). However,the code below is the explanation of our model.